# Run Simulations

## Import relevant libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

from configparser import ConfigParser

from typing import Callable

import gym
import gym_anytrading

from gym_anytrading.envs import CryptoEnvLogBLSH

from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3 import A2C, PPO, DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed

import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import os
import datetime

import multiprocessing

import quantstats as qs

In [2]:
# gpu = len(tf.config.list_physical_devices('GPU'))>0
# print("GPU is", "available" if gpu else "NOT AVAILABLE")

In [3]:
#multiprocessing.cpu_count()

## Get values from the config file

In [4]:
configur = ConfigParser()
config_file_name = "config_03_mix.ini"
print (configur.read(os.path.join('../config_files', config_file_name)))

['../config_files/config_03_mix.ini']


In [5]:
config_file_name_without_extension = config_file_name.replace(".ini", "")

ccy = configur.get('data', 'ccy')
data_frequency_train = configur.get('data', 'data_frequency_train')
data_frequency_val = configur.get('data', 'data_frequency_val')

window_size = configur.getint('environment', 'window_size')
start_date = configur.get('environment', 'start_date')
mid_date = configur.get('environment', 'mid_date')
end_date = configur.get('environment', 'end_date')
list_features = configur.get('environment', 'features').split(',')
target = configur.get('environment', 'target')

num_of_simulations = configur.getint('simulation', 'num_of_simulations')

In [6]:
def get_and_process_data(ccy, data_frequency):
    try:
        
        # Get the data
        filename_data = 'binance_' + ccy + "_" + data_frequency + '_from_2017_01_01_to_2022_12_31_candlesticks_signals_processed_technical_indicators_and_crypto_index_log.csv'
        fullpath_data = os.path.join('../../data/50_log', filename_data)
        df = pd.read_csv(fullpath_data)

        # Converting Date Column to DateTime Type
        # Set the index on the dataframe
        df['date_index'] = df['date']
        df.set_index('date_index', inplace=True)

        return df

    except Exception as e:
        print(e)

In [7]:
def my_process_data(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, target].to_numpy()[start:end]
    signal_features = env.df.loc[:, list_features].to_numpy()[start:end]
    dates = env.df.index.to_numpy()[start:end]
    return prices, signal_features, dates

class MyEnv(CryptoEnvLogBLSH):
    _process_data = my_process_data

In [8]:
df_train = get_and_process_data(ccy, data_frequency_train)
df_val = get_and_process_data(ccy, data_frequency_val)

In [9]:
print(f'df_train.shape: {df_train.shape}')
print(f'df_val.shape: {df_val.shape}')

df_train.shape: (187751, 95)
df_val.shape: (1842, 95)


In [10]:
df_train

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,SAR,ratio_high_open,ratio_low_open,ratio_close_open,time_to_chart_return,crypto_index,log_open,log_high,log_low,log_close
date_index,,,,,,,,,,,,,,,,,,,,,
2017-08-18 10:15:00,1503050400000,4304.15,4328.19,4296.04,4315.20,10.220599,1503051299999,4.401676e+04,50,8.507267,...,4252.425236,1.005585,0.998116,1.002567,0.002567,1002.567290,8.367335,8.372905,8.365449,8.369899
2017-08-18 10:30:00,1503051300000,4315.20,4349.00,4315.20,4349.00,18.197220,1503052199999,7.877310e+04,53,14.679352,...,4255.455827,1.007833,1.000000,1.007833,0.007833,1010.420176,8.369899,8.377701,8.369899,8.377701
2017-08-18 10:45:00,1503052200000,4330.26,4371.52,4330.26,4371.42,13.938950,1503053099999,6.080736e+04,65,9.907872,...,4261.068477,1.009528,1.000000,1.009505,0.005155,1015.629102,8.373383,8.382866,8.373383,8.382843
2017-08-18 11:00:00,1503053100000,4371.42,4371.42,4317.91,4356.31,9.206906,1503053999999,3.999195e+04,59,6.601114,...,4269.904599,1.000000,0.987759,0.996543,-0.003457,1012.118537,8.382843,8.382843,8.370527,8.379381
2017-08-18 11:15:00,1503054000000,4356.31,4357.37,4327.00,4353.00,8.332368,1503054899999,3.625639e+04,49,7.231167,...,4278.033831,1.000243,0.993272,0.999240,-0.000760,1011.349512,8.379381,8.379624,8.372630,8.378621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 23:00:00,1672526700000,16538.37,16538.84,16470.00,16520.81,2203.344040,1672527599999,3.637589e+07,49175,986.798280,...,16565.620000,1.000028,0.995866,0.998938,-0.001062,3838.344389,9.713438,9.713467,9.709296,9.712376
2022-12-31 23:15:00,1672527600000,16520.28,16524.91,16487.74,16502.67,1881.981230,1672528499999,3.106190e+07,52709,916.356800,...,16553.300000,1.000280,0.998030,0.998934,-0.001098,3834.129851,9.712344,9.712624,9.710372,9.711277
2022-12-31 23:30:00,1672528500000,16502.67,16529.54,16500.73,16524.64,1183.467290,1672529399999,1.955124e+07,38396,591.185520,...,16538.840000,1.001628,0.999882,1.001331,0.001331,3839.234227,9.711277,9.712904,9.711160,9.712608


In [11]:
start_date_id_train = int(df_train.index.get_loc(start_date))
mid_date_id_train = int(df_train.index.get_loc(mid_date))
mid_date_id_val = int(df_val.index.get_loc(mid_date))
end_date_id_val = int(df_val.index.get_loc(end_date))

In [12]:
def make_env(df, frame_bound, window_size, env_id: str, rank: int, seed: int = 0) -> Callable:
    """
    Utility function for multiprocessed env.
    
    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environment you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    :return: (Callable)
    """
    def _init() -> gym.Env:
        env = MyEnv(df=df, frame_bound=frame_bound, window_size=window_size)
        env.seed(seed + rank)
        return env
    set_random_seed(seed)
    return _init

In [13]:
list_models = ['A2C', 'PPO', 'DQN', 'RANDOM']

path_tensorboard = os.path.join("tensorboard", config_file_name.replace(".ini", ""))

df_sim_results = pd.DataFrame()

num_cpu = 8  # Number of processes to use
env_id = 'CryptoEnvLogBLSH-v1'

#setting up our environment for training 
env = SubprocVecEnv([make_env(df_train, (start_date_id_train, mid_date_id_train), window_size, env_id, i) for i in range(num_cpu)])

sim_id = 1

for model_name in list_models:

    if(model_name == 'A2C'):
        model = A2C('MlpPolicy', env, verbose=0, tensorboard_log=path_tensorboard) 
    elif(model_name == 'PPO'):
        model = PPO('MlpPolicy', env, batch_size=1024, verbose=0, tensorboard_log=path_tensorboard)
    elif(model_name == 'DQN'):
        model = DQN('MlpPolicy', env, batch_size=1024, verbose=0, tensorboard_log=path_tensorboard)

    #setting the learning timesteps
    model.learn(total_timesteps=(mid_date_id_train - start_date_id_train))
    
    # Export the model components
    
    if(model_name == 'A2C'):
        model.save(os.path.join('../save_models_components', 'model', 'model_' + model_name + "_" + config_file_name_without_extension))
        model.policy.save(os.path.join('../save_models_components', 'policy', 'policy_' + model_name + "_" + config_file_name_without_extension))
    elif(model_name == 'PPO'):
        model.save(os.path.join('../save_models_components', 'model', 'model_' + model_name + "_" + config_file_name_without_extension))
        model.policy.save(os.path.join('../save_models_components', 'policy', 'policy_' + model_name + "_" + config_file_name_without_extension))
    elif(model_name == 'DQN'):
        model.save(os.path.join('../save_models_components', 'model', 'model_' + model_name + "_" + config_file_name_without_extension))
        model.save_replay_buffer(os.path.join('../save_models_components', 'replay_buffer', 'replay_buffer_' + model_name + "_" + config_file_name_without_extension))
        model.policy.save(os.path.join('../save_models_components', 'policy', 'policy_' + model_name + "_" + config_file_name_without_extension))

    df_sim_results_temp = pd.DataFrame(columns=['total_reward_cash', 'total_profit_percentage', 'fmt_total_profit_percentage', 'num_of_trades'])

    list_sim_id = []

    for i in range(num_of_simulations):   
        
        list_sim_id.append(sim_id)

        # Check the progress
        if(i % 100 == 0):
            print(f'model_name:{model_name} - sim_id:{sim_id}')

        env = MyEnv(df=df_val, frame_bound=(mid_date_id_val,end_date_id_val), window_size=window_size)

        #Setting up the Agent Environment
        obs = env.reset()
        
        while True: 
            obs = obs[np.newaxis, ...]

            if(model_name == 'RANDOM'):
                action = env.action_space.sample()
            else:
                action, _states = model.predict(obs)
            
            obs, rewards, done, info = env.step(action)
            
            if done:
                df_sim_results_temp = df_sim_results_temp.append(info, ignore_index=True, sort=False)
                break

        # Export robot actions plot and data
        fig, df_robot_actions = env.render_all()
        # fig.write_html(os.path.join('results', 'plots', 'robot_actions', 'robot_actions_' + config_file_name_without_extension + '_sim_id_' + str(sim_id) + '.html'))

        # Export the history of details (the info dict inside of the environment)
        df_sim_env_data = pd.DataFrame.from_dict(env.history)

        # Merge robot actions data + sim env data
        df_sim_env_data_initial = pd.DataFrame(columns=['total_reward_cash', 'total_profit_percentage', 'fmt_total_profit_percentage', 'num_of_trades'])
        for i in range(0, window_size + 1):
            df_sim_env_data_initial = df_sim_env_data_initial.append(pd.Series([0, 0, 0, 0], index=df_sim_env_data_initial.columns), ignore_index=True)
        df_sim_env_data_temp = df_sim_env_data_initial.append(df_sim_env_data, ignore_index=True)
        df_robot_actions_and_env = pd.merge(df_robot_actions, df_sim_env_data_temp, left_index=True, right_index=True, how = "outer")
        
        # Export the Robot actions & env data
        file_export_robot_actions_and_env_results = config_file_name.replace(".ini", ".csv")
        fullpath_export_robot_actions_and_env_results = os.path.join('../results', 'data', 'robot_actions_and_env', 'robot_actions_and_env_results_' + config_file_name_without_extension + '_sim_id_' + str(sim_id) + ".csv")        
        df_robot_actions_and_env.to_csv(fullpath_export_robot_actions_and_env_results, index=False)

        # # Export quantstats
        # qs.extend_pandas()
        # net_worth = pd.Series(env.history['total_profit_percentage'], index=df_val.index[mid_date_id_val+1:end_date_id_val])
        # returns = net_worth.pct_change().iloc[1:]
        # qs.reports.html(returns, output='quantstats_' + config_file_name_without_extension + '_sim_id_' + str(sim_id) + '.html', title=config_file_name_without_extension + '_sim_id_' + str(sim_id), download_filename=os.path.join('results', 'plots', 'quantstats', 'quantstats_' + config_file_name_without_extension + '_sim_id_' + str(sim_id) + '.html'))

        sim_id = sim_id + 1
        
    df_sim_results_temp['sim_id'] = list_sim_id
    df_sim_results_temp['ccy'] = ccy
    df_sim_results_temp['data_frequency_train'] = data_frequency_train
    df_sim_results_temp['data_frequency_val'] = data_frequency_val
    df_sim_results_temp['window_size'] = window_size
    df_sim_results_temp['start_date'] = start_date
    df_sim_results_temp['mid_date'] = mid_date
    df_sim_results_temp['end_date'] = end_date
    df_sim_results_temp['list_features'] = str(list_features)
    df_sim_results_temp['model_type'] = model_name 
    df_sim_results_temp['num_of_simulations'] = num_of_simulations   

    df_sim_results = pd.concat([df_sim_results, df_sim_results_temp], axis=0)

model_name:A2C - sim_id:1
model_name:A2C - sim_id:101
model_name:A2C - sim_id:201
model_name:A2C - sim_id:301
model_name:A2C - sim_id:401
model_name:A2C - sim_id:501
model_name:A2C - sim_id:601
model_name:A2C - sim_id:701
model_name:A2C - sim_id:801
model_name:A2C - sim_id:901
model_name:PPO - sim_id:1001
model_name:PPO - sim_id:1101
model_name:PPO - sim_id:1201
model_name:PPO - sim_id:1301
model_name:PPO - sim_id:1401
model_name:PPO - sim_id:1501
model_name:PPO - sim_id:1601
model_name:PPO - sim_id:1701
model_name:PPO - sim_id:1801
model_name:PPO - sim_id:1901
model_name:DQN - sim_id:2001
model_name:DQN - sim_id:2101
model_name:DQN - sim_id:2201
model_name:DQN - sim_id:2301
model_name:DQN - sim_id:2401
model_name:DQN - sim_id:2501
model_name:DQN - sim_id:2601
model_name:DQN - sim_id:2701
model_name:DQN - sim_id:2801
model_name:DQN - sim_id:2901
model_name:RANDOM - sim_id:3001
model_name:RANDOM - sim_id:3101
model_name:RANDOM - sim_id:3201
model_name:RANDOM - sim_id:3301
model_name:RAN

## Results Analysis

### Pre-process results

In [14]:
df_sim_results = df_sim_results.reset_index(drop=True)

In [15]:
df_sim_results = df_sim_results.drop_duplicates()

### Display the results

### Plots

In [16]:
sns.set(rc={"figure.figsize":(15, 10)})

str_title = f"ccy:{ccy} | data_frequency_train:{data_frequency_train} | data_frequency_val:{data_frequency_val} | window_size:{window_size}<br>start_run_date:{mid_date} | end_run_date:{end_date} | lenght list_features:{len(list_features)} | num_of_simulations:{num_of_simulations}"

file_export_plot_sim_results = config_file_name.replace(".ini", "")

In [17]:
fig = px.box(df_sim_results, y="fmt_total_profit_percentage", color="model_type", points="all", color_discrete_sequence=[ "#FF7F0E", "#00CC96", "#10aded", "#8A56EF"],  width=800, height=600)
fig.update_layout(title=str_title, font={'size': 8})
fig.write_html(os.path.join('../results', 'plots', 'total_profit_percentage', 'box_plot_' + file_export_plot_sim_results + '.html'))
fig.show()

In [18]:
fig = px.histogram(df_sim_results, x="fmt_total_profit_percentage", color="model_type", color_discrete_sequence=[ "#FF7F0E", "#00CC96", "#10aded", "#8A56EF"],  width=800, height=600, marginal="rug", # can be `box`, `violin`
                         hover_data=['total_reward_cash','fmt_total_profit_percentage', 'num_of_trades', 'sim_id'])
fig.update_layout(title=str_title, font={'size': 8})
fig.write_html(os.path.join('../results', 'plots', 'total_profit_percentage', 'hist_plot_' + file_export_plot_sim_results + '.html'))
fig.show()

In [19]:
fig = px.histogram(df_sim_results, x="num_of_trades", color="model_type", color_discrete_sequence=[ "#FF7F0E", "#00CC96", "#10aded", "#8A56EF"],  width=800, height=600, marginal="rug", # can be `box`, `violin`
                         hover_data=['total_reward_cash','fmt_total_profit_percentage', 'num_of_trades', 'sim_id'])
fig.update_layout(title=str_title, font={'size': 8})
fig.write_html(os.path.join('../results', 'plots', 'num_of_trades', 'hist_plot_' + file_export_plot_sim_results + '.html'))
fig.show()

## Export Simul Results

In [20]:
file_export_sim_results = config_file_name.replace(".ini", ".csv")
fullpath_export_sim_results = os.path.join('../results', 'data', 'simul_results', 'sim_results_' + file_export_sim_results)

In [21]:
df_sim_results.to_csv(fullpath_export_sim_results, index=False)